### Step 1: Generate a Random Starting Solution
Implement a function to generate a random solution for the problem (e.g., random permutation for TSP).
### Step 2: Define Neighborhood Operations
- Define inter-route and intra-route moves (e.g., edge swaps, 2-opt, or others depending on the problem context).
- Ensure that edge reversals and their effect on evaluations are considered.
### Step 3: Evaluate Moves
Calculate move deltas for each move in the neighborhood.
Store improving moves in a list (let's call it LM).
### Step 4: Iterative Local Search with LM Management
For each iteration:
1. Check Moves in LM:
- If removed edges no longer exist: Remove the move from LM.
- If removed edges occur in a different relative direction: Retain the move in LM.
- If removed edges remain in the same direction: Apply the move, update the solution, and remove it from LM.
2. Evaluate New Moves:
- Add new improving moves to LM, considering inverted edges.
3. Update the Solution:
- Perform the best move from LM.

Repeat until no further improving moves exist.

# Evolutionary Computation Assignment 4

##### Solutions checked by solution checker

- Krzysztof Szala 144571
- Vadym Repetskyi 155610


In [ ]:
from utils import TspInstance, random_solution
import pandas as pd
import numpy as np

# Problem description

The goal of the task is to improve the time efficiency of the steepest local search with the use move
evaluations (deltas) from previous iterations (list of improving moves) using the neighborhood, which
turned out to be the best in assignment 3. Both inter-route and intra-route moves should be included
in the list. In the case of inter-route moves of the exchange of two edges, you should carefully read
the description of the lectures on the traveling salesman problem, in particular:
- Consider 3 situations (when we browse moves from LM):
    - Removed edges (defining the saved move) no longer exist in the current solution (at
    least one of them)
        - -> remove the move from LM
    - Removed edges occur in the current solution in a different relative direction from the
    saved one – not applicable now but the move can be applied in the future
        - -> leave the move in LM but do not apply it browse LM further
    - Removed edges appear in the current solution in the same relative direction (also
    both reversed)
        - -> perform (apply) the move and remove from LM
- When evaluating new moves we need to consider also moves with inverted edges


This mechanism should be used separately from candidate moves. Optionally, you can try to
implement them both together.
As starting solutions use random solutions.
As baseline report also results of the steepest local search with random starting solutions without
these mechanisms.

In [ ]:
def single_inter_move(tsp, solution, to_unselect, to_select, inter_type):
    solution = solution.copy()
    insert_index = np.where(solution == to_unselect)[0][0]

    prev = solution[insert_index - 1]
    next = solution[(insert_index + 1) % len(solution)]

    cost_change = (
        tsp.node_costs[to_select]
        + tsp.distance_matrix[prev, to_select]
        + tsp.distance_matrix[to_select, next]
        - tsp.node_costs[to_unselect]
        - tsp.distance_matrix[prev, to_unselect]
        - tsp.distance_matrix[to_unselect, next]
    )

    return cost_change, to_unselect, to_select, "inter", inter_type


def apply_inter_move(solution, to_unselect, to_select, inter_type):
    solution[np.where(solution == to_unselect)] = to_select
    return solution

In [ ]:
def single_intra_move(tsp, solution, a, b, intra_type):
    solution = solution.copy()
    index_a = np.where(solution == a)[0][0]
    index_b = np.where(solution == b)[0][0]

    if abs(index_a - index_b) == 1:
        return 0, a, b, "intra"
    
    if intra_type == "next":
        a_next = solution[(index_a + 1) % len(solution)]
        b_next = solution[(index_b + 1) % len(solution)]

        cost_change = (
            tsp.distance_matrix[a, b]
            + tsp.distance_matrix[a_next, b_next]
            - tsp.distance_matrix[a, a_next]
            - tsp.distance_matrix[b, b_next]
        )
    elif intra_type == "prev":
        a_prev = solution[(index_a - 1) % len(solution)]
        b_prev = solution[(index_b - 1) % len(solution)]

        cost_change = (
            tsp.distance_matrix[a, b]
            + tsp.distance_matrix[a_prev, b_prev]
            - tsp.distance_matrix[a_prev, a]
            - tsp.distance_matrix[b_prev, b]
        )

    return cost_change, a, b, "intra", intra_type

def apply_intra_move(solution, a, b, intra_type):
    index_a = np.where(solution == a)[0][0]
    index_b = np.where(solution == b)[0][0]
    solution = solution.copy()
    if intra_type == "next":
        if index_a < index_b:
            subsequence = solution[(index_a + 1) : index_b + 1][::-1]
            solution[(index_a + 1) : index_b + 1] = subsequence
        else:
            subsequence = solution[(index_b + 1) : index_a + 1][::-1]
            solution[(index_b + 1) : index_a + 1] = subsequence
            
    elif intra_type == "prev":
        if index_a < index_b:
            subsequence = solution[index_a : index_b][::-1]
            solution[index_a : index_b] = subsequence
        else:
            subsequence = solution[index_b : index_a][::-1]
            solution[index_b : index_a] = subsequence
    return solution

### Previous Results

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>A Candidate Edges Random</th>
      <th>B Candidate Edges Random</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>min_cost</th>
      <td>74938.000000</td>
      <td>47223.000000</td>
    </tr>
    <tr>
      <th>max_cost</th>
      <td>86066.000000</td>
      <td>52910.000000</td>
    </tr>
    <tr>
      <th>average_cost</th>
      <td>80740.440000</td>
      <td>49714.885000</td>
    </tr>
    <tr>
      <th>min_time</th>
      <td>0.544722</td>
      <td>0.562735</td>
    </tr>
    <tr>
      <th>max_time</th>
      <td>0.817196</td>
      <td>0.727541</td>
    </tr>
    <tr>
      <th>average_time</th>
      <td>0.625702</td>
      <td>0.638483</td>
    </tr>
  </tbody>
</table>
</div>

### Current Results

# Conclusions
- 
- 

[Source code: https://github.com/krzychuszala/EC-TSP](https://github.com/krzychuszala/EC-TSP)